# Wines Points prediction 

Submission Date : 3.6.2023
Task: Predict the wine score given the inputs
Instructions:
 * Use logistic regression as benchmark model
 * Use sklearn pipeliens + cv + grid search with sklearn models (e.g. KNNs, RandomForest, etc.)
 * Compare all models on proper metric (your choice)

For DNN course project:
* Use sklearn pipeliens with tensorflow models (w/wo embeddings, LSTMs, RNNs, Transformers etc.)
* Compare all models on proper metric (your choice)

In [1]:
%load_ext autoreload
%autoreload 2
import sys

Here we will try to predict the points a wine will get based on known characteristics (i.e. features, in the ML terminology). The mine point in this stage is to establish a simple, ideally super cost effective, basline.
In the real world there is a tradeoff between complexity and perforamnce, and the DS job, among others, is to present a tradeoff tables of what performance is achivalbel at what complexity level. 

to which models with increased complexity and resource demands will be compared. Complexity should then be translated into cost. For example:
 * Compute cost 
 * Maintenance cost
 * Serving costs (i.e. is new platform needed?) 
 

## Loading the data

In [2]:
import pandas as pd
import cufflinks as cf; cf.go_offline()

In [3]:
import os

print("Path at terminal when executing this file")
print(os.getcwd() + "\n")

Path at terminal when executing this file
C:\Users\yibar\Python_ML_2023\Exercises\Final Project



In [4]:
wine_reviews = pd.read_csv("winemag-data-130k-v2.csv", index_col = 0)
wine_reviews.shape

(129971, 13)

In [5]:
wine_reviews.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [6]:
wine_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 129971 entries, 0 to 129970
Data columns (total 13 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   country                129908 non-null  object 
 1   description            129971 non-null  object 
 2   designation            92506 non-null   object 
 3   points                 129971 non-null  int64  
 4   price                  120975 non-null  float64
 5   province               129908 non-null  object 
 6   region_1               108724 non-null  object 
 7   region_2               50511 non-null   object 
 8   taster_name            103727 non-null  object 
 9   taster_twitter_handle  98758 non-null   object 
 10  title                  129971 non-null  object 
 11  variety                129970 non-null  object 
 12  winery                 129971 non-null  object 
dtypes: float64(1), int64(1), object(11)
memory usage: 13.9+ MB


In [7]:
#set points column to float
wine_reviews.points = wine_reviews.points.astype('float64')

In [8]:
#Colunm 'taster_twitter_handle' has no added value, same as taster_name
wine_reviews.drop('taster_twitter_handle', axis=1, inplace=True)

In [9]:
wine_reviews.describe(include = 'all').transpose()

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
country,129908,43,US,54504,NaN,NaN,NaN,NaN,NaN,NaN,NaN
description,129971,119955,"Seductively tart in lemon pith, cranberry and ...",3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
designation,92506,37979,Reserve,2009,NaN,NaN,NaN,NaN,NaN,NaN,NaN
points,129971.0,NaN,NaN,NaN,88.447138,3.03973,80.0,86.0,88.0,91.0,100.0
price,120975.0,NaN,NaN,NaN,35.363389,41.022218,4.0,17.0,25.0,42.0,3300.0
province,129908,425,California,36247,NaN,NaN,NaN,NaN,NaN,NaN,NaN
region_1,108724,1229,Napa Valley,4480,NaN,NaN,NaN,NaN,NaN,NaN,NaN
region_2,50511,17,Central Coast,11065,NaN,NaN,NaN,NaN,NaN,NaN,NaN
taster_name,103727,19,Roger Voss,25514,NaN,NaN,NaN,NaN,NaN,NaN,NaN
title,129971,118840,Gloria Ferrer NV Sonoma Brut Sparkling (Sonoma...,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# Fill null values

In [11]:
wine_reviews.isnull().sum().sort_values(ascending=False)

region_2       79460
designation    37465
taster_name    26244
region_1       21247
price           8996
country           63
province          63
variety            1
description        0
points             0
title              0
winery             0
dtype: int64

In [12]:
# When country is null, try to fill from the counry of the same winery from the dataset
null_winery = pd.Series(wine_reviews[wine_reviews.country.isnull()].winery)

In [13]:
ds_winery_country = wine_reviews.loc[wine_reviews.winery.isin(null_winery.values), ['winery','country']] \
    .sort_values(['winery', 'country']).fillna(method = 'ffill').drop_duplicates()
ds_winery_country.columns = ['winery', 'country1']
ds_winery_country.head()

,winery,country1
119787,Amiran Vepkhvadze,NaN
13062,Bachelder,Canada
34853,Bartho Eksteen,South Africa
2789,Barton & Guestier,France
129590,Büyülübağ,France


In [14]:
wine_reviews = wine_reviews.merge(ds_winery_country, on = 'winery', how = 'left')
wine_reviews.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,title,variety,winery,country1
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87.0,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,NaN
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87.0,15.0,Douro,NaN,NaN,Roger Voss,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,NaN
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87.0,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,NaN
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87.0,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,NaN
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87.0,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,NaN


In [15]:
wine_reviews.loc[wine_reviews.country.isnull() & wine_reviews.country1.notnull() ,'country'] = \
    wine_reviews.loc[wine_reviews.country.isnull() & wine_reviews.country1.notnull() ,'country1']

wine_reviews.drop('country1', axis=1, inplace=True)

In [16]:
wine_reviews.loc[wine_reviews.country.isnull()]

,country,description,designation,points,price,province,region_1,region_2,taster_name,title,variety,winery
119787,NaN,This deep red-violet wine has a nose of blackb...,NaN,90.0,30.0,NaN,NaN,NaN,Mike DeSimone,Amiran Vepkhvadze 2014 Otskhanuri Sapere,Otskhanuri Sapere,Amiran Vepkhvadze


In [17]:
#Only one row left without country. The correct country for The last winery is Georgia, according to google...
wine_reviews.loc[wine_reviews.country.isnull(), 'country'] = 'Georgia'

In [18]:
wine_reviews.isnull().sum().sort_values(ascending=False)

region_2       79460
designation    37465
taster_name    26244
region_1       21247
price           8996
province          63
variety            1
country            0
description        0
points             0
title              0
winery             0
dtype: int64

In [19]:
#Replace all text null values by "Unknown" word
wine_reviews.loc[:, wine_reviews.columns != 'price'] = wine_reviews.loc[:, wine_reviews.columns != 'price'].fillna(value = 'Unknown')

In [20]:
wine_reviews.isnull().sum().sort_values(ascending=False)

price          8996
country           0
description       0
designation       0
points            0
province          0
region_1          0
region_2          0
taster_name       0
title             0
variety           0
winery            0
dtype: int64

In [21]:
#Replace null prices according to avarage price of:
#1. winery / variety / designation
#2. winery / variety
#3. winery 
#4. total avarage
wine_reviews['price'].fillna(wine_reviews.groupby(['winery', 'variety', 'designation'])['price'].transform('mean'), inplace = True)
wine_reviews['price'].fillna(wine_reviews.groupby(['winery', 'variety'])['price'].transform('mean'), inplace = True)
wine_reviews['price'].fillna(wine_reviews.groupby(['winery'])['price'].transform('mean'), inplace = True)
wine_reviews['price'].fillna(wine_reviews.price.mean(), inplace = True)

In [22]:
wine_reviews['price'].isnull().sum()

0

In [23]:
wine_reviews.describe()

,points,price
count,129971.000000,129971.000000
mean,88.447138,36.202300
std,3.039730,45.746672
min,80.000000,4.000000
25%,86.000000,17.000000
50%,88.000000,26.000000
75%,91.000000,42.000000
max,100.000000,3300.000000


In [24]:
def extact_year(title):
    for word in title.split():
        if word.isdigit():
            if 1900 <= int(word) <= 2020:
                return word
        elif word == 'NV':
            return word
    return None

In [25]:
#add Year Column
wine_reviews['year'] = wine_reviews['title'].apply(lambda i: extact_year(i))
wine_reviews.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,title,variety,winery,year
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87.0,19.0,Sicily & Sardinia,Etna,Unknown,Kerin O’Keefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,2013
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87.0,15.0,Douro,Unknown,Unknown,Roger Voss,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,2011
2,US,"Tart and snappy, the flavors of lime flesh and...",Unknown,87.0,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,2013
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87.0,13.0,Michigan,Lake Michigan Shore,Unknown,Alexander Peartree,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,2013
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87.0,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,2012


In [26]:
wine_reviews.isnull().sum().sort_values(ascending=False)

country        0
description    0
designation    0
points         0
price          0
province       0
region_1       0
region_2       0
taster_name    0
title          0
variety        0
winery         0
year           0
dtype: int64

In [27]:
wine_reviews['year'].drop_duplicates().sort_values()

9728      1912
7986      1929
107854    1934
41441     1935
90583     1941
117205    1945
51898     1947
82827     1952
78299     1957
78312     1961
52636     1963
12860     1964
45034     1965
78310     1966
16331     1967
46010     1968
36196     1969
45000     1973
90603     1974
12530     1976
45010     1978
31417     1980
60004     1982
16011     1983
78311     1984
54667     1985
60384     1986
31425     1987
18792     1988
33974     1989
18791     1990
370       1991
6050      1992
33976     1993
6037      1994
2040      1995
378       1996
375       1997
4731      1998
367       1999
366       2000
132       2001
243       2002
120       2003
119       2004
137       2005
123       2006
22        2007
50        2008
24        2009
16        2010
1         2011
4         2012
0         2013
59        2014
61        2015
77        2016
1618      2017
63          NV
Name: year, dtype: object

In [28]:
#Adding log of price to reduce outlier prices
#Therefore, we can remove it.
import numpy as np
wine_reviews['price_log'] = np.log(wine_reviews.price)
wine_reviews.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,title,variety,winery,year,price_log
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87.0,19.0,Sicily & Sardinia,Etna,Unknown,Kerin O’Keefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,2013,2.944439
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87.0,15.0,Douro,Unknown,Unknown,Roger Voss,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,2011,2.708050
2,US,"Tart and snappy, the flavors of lime flesh and...",Unknown,87.0,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,2013,2.639057
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87.0,13.0,Michigan,Lake Michigan Shore,Unknown,Alexander Peartree,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,2013,2.564949
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87.0,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,2012,4.174387


In [29]:
wine_reviews.describe(include = 'all').transpose()

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
country,129971,43,US,54504,NaN,NaN,NaN,NaN,NaN,NaN,NaN
description,129971,119955,"Seductively tart in lemon pith, cranberry and ...",3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
designation,129971,37980,Unknown,37465,NaN,NaN,NaN,NaN,NaN,NaN,NaN
points,129971.0,NaN,NaN,NaN,88.447138,3.03973,80.0,86.0,88.0,91.0,100.0
price,129971.0,NaN,NaN,NaN,36.2023,45.746672,4.0,17.0,26.0,42.0,3300.0
province,129971,426,California,36247,NaN,NaN,NaN,NaN,NaN,NaN,NaN
region_1,129971,1230,Unknown,21247,NaN,NaN,NaN,NaN,NaN,NaN,NaN
region_2,129971,18,Unknown,79460,NaN,NaN,NaN,NaN,NaN,NaN,NaN
taster_name,129971,20,Unknown,26244,NaN,NaN,NaN,NaN,NaN,NaN,NaN
title,129971,118840,Gloria Ferrer NV Sonoma Brut Sparkling (Sonoma...,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN
